## Connect to Spark standalone cluster

In [ ]:
!/usr/local/spark/sbin/stop-all.sh

In [ ]:
!/usr/local/spark/sbin/start-all.sh

In [ ]:
import os
import pyspark
import pymssql
import pandas as pd
from pyspark import SparkContext, SparkConf, pandas as ps
from pyspark.sql import SparkSession


In [ ]:
#Load PG JDBC driver when starting Spark
sparkClassPath =  '/usr/local/spark/jars/postgresql-42.4.1.jar'

In [ ]:
try:
    spark.stop()
except:
    print("No Spark Session")
    
spark = SparkSession.builder \
        .config("spark.driver.extraClassPath", sparkClassPath) \
        .config("spark.jars", sparkClassPath) \
        .appName("Southbridge Analytics") \
        .master("local") \
        .getOrCreate()


## Connect to PostgreSQL Server and create DataFrame

In [ ]:
url = "jdbc:postgresql:pg"
user = "postgres"
password = "P@ssw0rd"
table = "dvdrental.film"

jdbcDF1 = spark.read.format("jdbc") \
        .option("url", "jdbc:postgresql://pg:5432/dvdrental") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", "film") \
        .option("user", "postgres").option("password", "P@ssw0rd").load()

jdbcDF2 = spark.read.format("jdbc") \
        .option("url", "jdbc:postgresql://pg:5432/dvdrental") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", "address") \
        .option("user", "postgres").option("password", "P@ssw0rd").load()



In [ ]:
jdbcDF1.show(5)
jdbcDF2.show(5)

## Write DataFrame to HDFS as Parquet

In [ ]:
# create dir in hdfs if not already there
os.system('hdfs dfs -mkdir hdfs://localhost:9000/pg-spoke/')

In [ ]:
jdbcDF1.write.parquet("hdfs://localhost:9000/pg-spoke/pg1.parquet")
jdbcDF2.write.parquet("hdfs://localhost:9000/pg-spoke/pg2.parquet")

## Validate Parquet from HDFS

In [ ]:
testdf1 = spark.read.parquet("hdfs://localhost:9000/pg-spoke/pg1.parquet")
testdf2 = spark.read.parquet("hdfs://localhost:9000/pg-spoke/pg2.parquet")

In [ ]:
testdf1.show(2)
testdf2.show(2)